In [1]:
using BenchmarkTools
using StaticArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "3090298014686060716", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "8998502618071127596", 2, "feature")

## Vecchia funzione

In [6]:
function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	for i=1:length(obj.body)
		if isa(obj.body[i],Matrix)
			mat = obj.body[i]
			d,d = size(mat)
			newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
			for h in range(1, length=d)
				for k in range(1, length=d)
					newMat[h,k]=mat[h,k]
				end
			end
			push!(cloned.body,[newMat])
		elseif (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==3
			V,FV,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,FV,EV)])
		elseif  (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==2
			V,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,EV)])
		elseif isa(obj.body[i],Lar.Struct)
			newObj = Struct()
			newObj.box = [ [obj.body[i].box[1];zeros(dimadd)],
							[obj.body[i].box[2];zeros(dimadd)] ]
			newObj.category = (obj.body[i]).category
			push!(cloned.body,[embedTraversal(newObj,obj.body[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 1 method)

In [7]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  8.833 μs (25 allocations: 1.81 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "9958981140883169293", Any, "feature")

In [8]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  33.900 μs (87 allocations: 4.61 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "12204753819417774865", Any, "feature")

In [9]:
@code_llvm embedTraversal(Lar.Struct(),structure,1,"New")


;  @ In[6]:1 within `embedTraversal'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_embedTraversal_2152(%jl_value_t* nonnull align 8 dereferenceable(40), %jl_value_t* nonnull align 8 dereferenceable(40), i64, %jl_value_t* nonnull) #0 {
top:
  %4 = alloca %jl_value_t*, i32 3
  %gcframe = alloca %jl_value_t*, i32 9, align 16
  %5 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %5, i8 0, i32 72, i1 false)
  %6 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #10
;  @ In[6]:2 within `embedTraversal'
; ┌ @ Base.jl:33 within `getproperty'
   %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
   %8 = bitcast %jl_value_t** %7 to i64*
   store i64 28, i64* %8
   %9 = getelementptr %jl_value_t**, %jl_value_t*** %6, i32 0
   %10 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
   %11 = bitcast %jl_value_t** %10 to %jl_value_t***
   %12 = load %jl_value_t**, %jl_value_t*** %9
   store %jl_value_t** 

In [10]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  4.61 KiB
  allocs estimate:  87
  --------------
  minimum time:     33.200 μs (0.00% GC)
  median time:      43.700 μs (0.00% GC)
  mean time:        47.568 μs (1.16% GC)
  maximum time:     5.717 ms (96.80% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Nuove funzioni

"""\
embedTraversalMatrix(objBody::Matrix,n::Int)::Matrix\
Funzione chiamata se il corpo della struttura che si sta modellando è una matrice. Trasforma la matrice aumentandone la dimensione del valore n.\
"""

In [11]:
function embedTraversalMatrix(objBody)
	mat = objBody[i]
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	for h in range(1, length=d)
		for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 1 method)

"""\
embedTraversalTupleOrArray(objBody::Union{Tuple,Array},V)\
Funzione chiamata se il corpo della struttura che si sta modellando è una tupla o un array. Genera e restituisce il parametro V.\
"""

In [12]:
function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

"""\
embedTraversalStruct(objBody::Struct)::Struct\
Funzione chiamata se il corpo della struttura che si sta modellando è una struttura. Genera(a partire dalla struttura in ingresso objBody) e restituisce, la struttura newObj.\
"""

In [13]:
function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody[i].box[1];zeros(dimadd)],
				   [objBody[i].box[2];zeros(dimadd)] ]
	newObj.category = (objBody[i]).category
	return newObj
end

embedTraversalStruct (generic function with 1 method)

"""\
	embedTraversal(cloned::Struct,obj::Struct,n::Int,suffix::String)::Struct\
Restituisce l’oggetto cloned di tipo “struct” che consiste in una struttura avente
lo stesso body della struttura “obj” passata come parametro. Sono visitati uno
per uno gli elementi del body di obj e siano essi matrici, tuple, array o a loro
volta strutture, sono copiati all’interno di variabili locali e inserite nella struttura
“cloned”.\
"""

In [14]:
function embedTraversal(cloned,obj,n,suffix)
	objBody = obj.body
	for i=1:length(objBody)
		if isa(objBody[i],Matrix)
			newMat = embedTraversalMatrix(objBody)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody)
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [15]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  11.001 μs (25 allocations: 1.80 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "4028633255938764952", Any, "feature")

In [16]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  26.201 μs (87 allocations: 4.63 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "1141402712469613185", Any, "feature")

In [17]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  4.61 KiB
  allocs estimate:  87
  --------------
  minimum time:     27.400 μs (0.00% GC)
  median time:      52.599 μs (0.00% GC)
  mean time:        72.106 μs (1.15% GC)
  maximum time:     9.810 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [18]:
@benchmark embedTraversal(Lar.Struct(),x,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  1.80 KiB
  allocs estimate:  25
  --------------
  minimum time:     13.400 μs (0.00% GC)
  median time:      26.400 μs (0.00% GC)
  mean time:        28.523 μs (0.00% GC)
  maximum time:     4.482 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [19]:
@code_typed embedTraversal(Lar.Struct(),structure,1,"New")

CodeInfo(
1 ── %1   = Base.getfield(obj, :body)::Array
│    %2   = Base.arraylen(%1)::Int64
│    %3   = Base.sle_int(1, %2)::Bool
│    %4   = Base.ifelse(%3, %2, 0)::Int64
│    %5   = Base.slt_int(%4, 1)::Bool
└───        goto #3 if not %5
2 ──        goto #4
3 ──        goto #4
4 ┄─ %9   = φ (#2 => true, #3 => false)::Bool
│    %10  = φ (#3 => 1)::Int64
│    %11  = φ (#3 => 1)::Int64
│    %12  = Base.not_int(%9)::Bool
└───        goto #63 if not %12
5 ┄─ %14  = φ (#4 => %10, #62 => %217)::Int64
│    %15  = φ (#4 => %11, #62 => %218)::Int64
│    %16  = Base.getfield(obj, :body)::Array
│    %17  = Base.getindex(%16, %14)::Any
│    %18  = (%17 isa Main.Matrix)::Bool
└───        goto #40 if not %18
6 ── %20  = Base.getfield(obj, :body)::Array
│    %21  = Base.getindex(%20, %14)::Any
│    %22  = Main.size(%21)::Any
│    %23  = Base.indexed_iterate(%22, 1)::Any
│           Core.getfield(%23, 1)::Any
│    %25  = Core.getfield(%23, 2)::Any
│    %26  = Base.indexed_iterate(%22, 2, %25)::Any
│ 

In [20]:
@code_warntype embedTraversal(Lar.Struct(),x,1,"New")

Variables
  #self#::Core.Compiler.Const(embedTraversal, false)
  cloned::LinearAlgebraicRepresentation.Struct
  obj::LinearAlgebraicRepresentation.Struct
  n::Int64
  suffix::String
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  @_8::Any
  @_9::Any
  @_10::Any
  mat::Any
  d::Any
  newMat::Array{Float64,2}
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  V::Any
  FV::Any
  EV::Any
  dimadd::Int64
  ncols::Any
  nmat::Array{Float64,2}
  newObj::Any
  h::Int64
  @_23::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_25::Bool
  @_26::Any
  @_27::Bool
  @_28::Any

Body::LinearAlgebraicRepresentation.Struct
1 ── %1   = Base.getproperty(obj, :body)::Array
│    %2   = Main.length(%1)::Int64
│    %3   = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│           (@_6 = Base.iterate(%3))
│    %5   = (@_6 === nothing)::Bool
│    %6   = Base.not_int(%5)::Bool
└───        goto #30 if not %6
2 ┄─        Core.NewvarNode(:(@_8))
│           Core.N

│    %119 = Core.tuple(V, FV, EV)::Tuple{Any,Any,Any}
│    %120 = Base.vect(%119)::Array{_A,1} where _A
│           Main.push!(%118, %120)
└───        goto #28
18 ─ %123 = Base.getproperty(obj, :body)::Array
│    %124 = Base.getindex(%123, i)::Any
│    %125 = (%124 isa Main.Tuple)::Bool
└───        goto #20 if not %125
19 ─        (@_27 = %125)
└───        goto #21
20 ─ %129 = Base.getproperty(obj, :body)::Array
│    %130 = Base.getindex(%129, i)::Any
└───        (@_27 = %130 isa Main.Array)
21 ┄        goto #23 if not @_27
22 ─ %133 = Base.getproperty(obj, :body)::Array
│    %134 = Base.getindex(%133, i)::Any
│    %135 = Main.length(%134)::Any
│           (@_28 = %135 == 2)
└───        goto #24
23 ─        (@_28 = false)
24 ┄        goto #26 if not @_28
25 ─ %140 = Base.getproperty(obj, :body)::Array
│    %141 = Base.getindex(%140, i)::Any
│    %142 = Main.deepcopy(%141)::Any
│    %143 = Base.indexed_iterate(%142, 1)::Any
│           (V = Core.getfield(%143, 1))
│           (@_10 = Co

## Modifiche

In [21]:
@inline function embedTraversalMatrix(objBody,n)
	mat = objBody
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	@inbounds for h in range(1, length=d)
		@inbounds for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 2 methods)

In [22]:
@inline function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

In [23]:
@inline function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody.box[1];zeros(dimadd)],
				   [objBody.box[2];zeros(dimadd)] ]
	newObj.category = objBody.category
	return newObj
end

embedTraversalStruct (generic function with 1 method)

In [24]:
@inline function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	objBody = obj.body
	@inbounds @async for i=1:length(objBody)
		if (isa(objBody[i],MMatrix) || isa(objBody[i],Matrix))
			newMat = embedTraversalMatrix(objBody[i],n)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody[i])
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [25]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  5.583 μs (9 allocations: 1.08 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "16680973628187021244", Any, "feature")

In [26]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  5.450 μs (9 allocations: 1.08 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "11114942229055723771", Any, "feature")

In [27]:
@code_llvm embedTraversal(Lar.Struct(),structure,1,"New")


;  @ In[24]:1 within `embedTraversal'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_embedTraversal_2948(%jl_value_t* nonnull align 8 dereferenceable(40), %jl_value_t* nonnull align 8 dereferenceable(40), i64, %jl_value_t* nonnull) #0 {
top:
  %4 = alloca %jl_value_t*, i32 6
  %gcframe = alloca %jl_value_t*, i32 5, align 16
  %5 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %5, i8 0, i32 40, i1 false)
  %6 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #8
;  @ In[24]:2 within `embedTraversal'
; ┌ @ Base.jl:33 within `getproperty'
   %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
   %8 = bitcast %jl_value_t** %7 to i64*
   store i64 12, i64* %8
   %9 = getelementptr %jl_value_t**, %jl_value_t*** %6, i32 0
   %10 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
   %11 = bitcast %jl_value_t** %10 to %jl_value_t***
   %12 = load %jl_value_t**, %jl_value_t*** %9
   store %jl_value_t**

In [28]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  1.08 KiB
  allocs estimate:  9
  --------------
  minimum time:     5.367 μs (0.00% GC)
  median time:      10.067 μs (0.00% GC)
  mean time:        11.470 μs (0.00% GC)
  maximum time:     612.784 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6